### Analysis of 2023 (entire year available in the official website of data.gouv.fr)

We import the file, then we check by 'chunk' (to not work too much the machine) the first lines in purpose to ensure validity

In [13]:
import pandas as pd

# path of the file (adjust it according to your computer)
file_path = 'C:/Users/NEZHA/OneDrive/Documents/DVF/ValeursFoncieres-2023.txt'

# Columns that we conserve to our little check, we will display their first lines 
cols_to_keep = ['Date mutation', 'Valeur fonciere', 'Code postal', 'Commune', 
                'Surface Carrez du 1er lot', 'Nombre de lots', 'Type local', 'Surface reelle bati']

# 100 000 lines at a time
chunksize = 100000
chunks = pd.read_csv(file_path, delimiter='|', encoding='utf-8', chunksize=chunksize, usecols=cols_to_keep, low_memory=False)

# We display five lines to check if the importation is valid
for chunk in chunks:
    print(chunk.head())
    break  # We stop after the first chunk... that's enough

  Date mutation Valeur fonciere  Code postal              Commune  \
0    05/01/2023      1070000,00       1630.0     ST-GENIS-POUILLY   
1    05/01/2023      1070000,00       1630.0     ST-GENIS-POUILLY   
2    05/01/2023      1070000,00       1630.0     ST-GENIS-POUILLY   
3    03/01/2023       152200,00       1450.0    SERRIERES-SUR-AIN   
4    05/01/2023       269000,00       1800.0  SAINT-JEAN-DE-NIOST   

  Surface Carrez du 1er lot  Nombre de lots   Type local  Surface reelle bati  
0                       NaN               1   Dépendance                  0.0  
1                       NaN               1  Appartement                233.0  
2                       NaN               1   Dépendance                  0.0  
3                       NaN               0       Maison                 64.0  
4                       NaN               0       Maison                 73.0  


it works perfectly ! let's keep going... Now we filter and keep the data which corresponds to the postal codes for the center of Paris metropolitan area (we consider that the center is Paris intra-muros), and we check the first lines...

In [14]:
import pandas as pd

# path (adjust it according to your computer)
file_path = 'C:/Users/NEZHA/OneDrive/Documents/DVF/ValeursFoncieres-2023.txt'

# 100 000 lines at a time
chunk_size = 100000

# List to stock and manage the chunks
df_paris_filtered = []

# we read it by chunks (with low memory to avoid advertissement)
chunks = pd.read_csv(file_path, delimiter='|', encoding='utf-8', chunksize=chunk_size, low_memory=False)

# we filter and add the filtered chunks
for chunk in chunks:
    # Filtering for center of Paris's agglomeration, Paris intra-muros (postal codes between 75000 and 75020) in the dataFrame
    df_paris_chunk = chunk[chunk['Code postal'].between(75000, 75020)]
    
    # we add the filtered chunk in the list
    df_paris_filtered.append(df_paris_chunk)

# we combine all chunks in only one DataFrame
df_paris = pd.concat(df_paris_filtered)

# checking of the first lines
print(df_paris.head())

# we display every columns
pd.set_option('display.max_columns', None)


         Identifiant de document  Reference document  1 Articles CGI  \
3507403                      NaN                 NaN             NaN   
3507404                      NaN                 NaN             NaN   
3507462                      NaN                 NaN             NaN   
3509871                      NaN                 NaN             NaN   
3511591                      NaN                 NaN             NaN   

         2 Articles CGI  3 Articles CGI  4 Articles CGI  5 Articles CGI  \
3507403             NaN             NaN             NaN             NaN   
3507404             NaN             NaN             NaN             NaN   
3507462             NaN             NaN             NaN             NaN   
3509871             NaN             NaN             NaN             NaN   
3511591             NaN             NaN             NaN             NaN   

         No disposition Date mutation Nature mutation Valeur fonciere  \
3507403               1    13/01/2023      

C:\Users\NEZHA\AppData\Local\Temp\ipykernel_7116\3761131506.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_paris = pd.concat(df_paris_filtered)


we create a column year to treat the data well

In [15]:
import pandas as pd

# Chemin du fichier
file_path2 = 'C:/Users/NEZHA/OneDrive/Documents/DVF/ValeursFoncieres-2023.txt'

# we limit at 100000
chunk_size = 100000

# List to treat chunks
df_paris_filtered = []

# read by chunks
chunks = pd.read_csv(file_path2, delimiter='|', encoding='utf-8', chunksize=chunk_size, low_memory=False)

# we filter, then convert the chunks treated
for chunk in chunks:
    # Filtering for center of Paris's agglomeration, Paris intra-muros (postal codes between 75000 and 75020)
    df_paris_chunk = chunk[chunk['Code postal'].between(75000, 75020)]

    # we convert the column 'Date mutation' in datetime
    df_paris_chunk['Date mutation'] = pd.to_datetime(df_paris_chunk['Date mutation'], errors='coerce')

    # Extraction after conversion
    df_paris_chunk['Année'] = df_paris_chunk['Date mutation'].dt.year

    # then we add the chunk treated
    df_paris_filtered.append(df_paris_chunk)

# we combine all chunks in one DataFrame
df_paris = pd.concat(df_paris_filtered)

# we check
print(df_paris[['Date mutation', 'Année']].head())

C:\Users\NEZHA\AppData\Local\Temp\ipykernel_7116\2758603801.py:21: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_paris_chunk['Date mutation'] = pd.to_datetime(df_paris_chunk['Date mutation'], errors='coerce')
C:\Users\NEZHA\AppData\Local\Temp\ipykernel_7116\2758603801.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_paris_chunk['Date mutation'] = pd.to_datetime(df_paris_chunk['Date mutation'], errors='coerce')
C:\Users\NEZHA\AppData\Local\Temp\ipykernel_7116\2758603801.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


        Date mutation   Année
3507403    2023-01-13  2023.0
3507404    2023-01-13  2023.0
3507462    2023-01-18  2023.0
3509871    2023-02-02  2023.0
3511591    2023-03-01  2023.0


C:\Users\NEZHA\AppData\Local\Temp\ipykernel_7116\2758603801.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_paris_chunk['Date mutation'] = pd.to_datetime(df_paris_chunk['Date mutation'], errors='coerce')
C:\Users\NEZHA\AppData\Local\Temp\ipykernel_7116\2758603801.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_paris_chunk['Année'] = df_paris_chunk['Date mutation'].dt.year
C:\Users\NEZHA\AppData\Local\Temp\ipykernel_7116\2758603801.py:30: FutureWarning: The behavior of DataFrame concatenation

In [16]:
# We create a copy of df_paris to avoid the view and work directly on the copy
df_paris_copy = df_paris.copy()

# We replace commas by points in the column 'Valeur fonciere' of the dataFrame
df_paris_copy['Valeur fonciere'] = df_paris_copy['Valeur fonciere'].replace({',': '.'}, regex=True)

# We convert 'Valeur fonciere' column to numeric, forcing errors to NaN
df_paris_copy['Valeur fonciere'] = pd.to_numeric(df_paris_copy['Valeur fonciere'], errors='coerce')

# we verify the type to check if it worked
print(df_paris_copy['Valeur fonciere'].dtype)  # it must display 'float64'

# We check the first lines
print(df_paris_copy[['Valeur fonciere']].head())

float64
         Valeur fonciere
3507403         250000.0
3507404         250000.0
3507462          22000.0
3509871          27000.0
3511591         339550.0


Little manipulation to see the some outliers

In [17]:
# we filter transaction that are superior to 1 000 000
high_value_transactions = df_paris_copy[df_paris_copy['Valeur fonciere'] > 1000000]

# we display the first lines
print(high_value_transactions[['Valeur fonciere']].head())

         Valeur fonciere
3524340        2329600.0
3524341        2329600.0
3524342        2329600.0
3524343        2329600.0
3529290        1343700.0


In [18]:
# We check if the 'surface réelle bati' is present and valid
df_paris_copy['Prix_m2'] = df_paris_copy['Valeur fonciere'] / df_paris_copy['Surface reelle bati']

# We filter for values of price per m² reasonable, we don't use quantiles because otherwise ou computer bugs, but it would be the best idea... 
df_paris_filtered_m2 = df_paris_copy[df_paris_copy['Prix_m2'] < 30000]  

# After filtering, we calcul again
prix_moyen_par_annee_m2 = df_paris_filtered_m2.groupby('Année')['Prix_m2'].mean()

# We display
print(prix_moyen_par_annee_m2)


Année
2023.0    10413.308109
Name: Prix_m2, dtype: float64
